In [1]:
import numpy as np
import string
from pickle import dump,load
from keras.models import Sequential
from keras.layers import LSTM,Dense,Embedding
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
#function used to clean the doc
def clean_doc(doc):
    doc = doc.replace('--', ' ')
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [3]:
filename="/home/hardik/Desktop/pluto_book_1.txt"

In [4]:
# opening file
file=open(filename,"r")
text=file.read()


In [5]:
#creating sequences 
tokens = clean_doc(text)
# organize into sequences of tokens
length = 50 + 1
sequences = []
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)

In [6]:
#saving clean file

data = '\n'.join(sequences)
file = open("/home/hardik/Desktop/sequences.txt", 'w')
file.write(data)
file.close()

In [7]:
#calling saved file

file=open("/home/hardik/Desktop/sequences.txt","r")
text=file.read()
file.close()
line=text.split("\n")

In [8]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(line)
sequence=tokenizer.texts_to_sequences(line)

In [9]:
vocab=len(tokenizer.word_index)+1

In [10]:
vocab

7410

In [11]:
sequence=np.array(sequence)

In [12]:
sequence

array([[  11, 1045,  329, ...,   57, 1265,   35],
       [1045,  329, 7409, ..., 1265,   35,    1],
       [ 329, 7409,    4, ...,   35,    1, 2874],
       ...,
       [ 382,  467,    4, ...,  414,   13,   21],
       [ 467,    4,   33, ...,   13,   21,   23],
       [   4,   33,   79, ...,   21,   23,   85]])

In [13]:
X, y = sequence[:,:-1], sequence[:,-1]


In [14]:
X

array([[  11, 1045,  329, ...,   11,   57, 1265],
       [1045,  329, 7409, ...,   57, 1265,   35],
       [ 329, 7409,    4, ..., 1265,   35,    1],
       ...,
       [ 382,  467,    4, ..., 1044,  414,   13],
       [ 467,    4,   33, ...,  414,   13,   21],
       [   4,   33,   79, ...,   13,   21,   23]])

In [15]:
y

array([  35,    1, 2874, ...,   21,   23,   85])

In [16]:
y = np_utils.to_categorical(y, num_classes=vocab)
seq_length = X.shape[1]

MemoryError: 

In [ ]:
# define model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100)
 
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [ ]:
# load the model
model = load_model('model.h5')
 
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
 

In [ ]:
result = []
in_text = input("enter text whose further word sentence you wanna predict:")
# generate a fixed number of words
for _ in range(n_words):
    # encode the text as integer
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    # truncate sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    # predict probabilities for each word
    yhat = model.predict_classes(encoded, verbose=0)
    # map predicted word index to word
    out_word = ''
    for word, index in tokenizer.word_index.items():
        if index == yhat:
            out_word = word
            break
# append to input
    in_text += ' ' + out_word
    result.append(out_word)
print(' '.join(result))